In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import shutil
import sys
import numpy as np
import pandas as pd
import tensorflow as tf

In [70]:
_CSV_COLUMNS = [
    '', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 
    'target', 'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
]

_CSV_COLUMN_DEFAULTS = [[0], [''], [''], [''], [''], [''], 
                        [''], [0.0], [''], [''], [''], [''], 
                        [0.0], [0], [0], [''], [0], [0], [0]]

parser = argparse.ArgumentParser()

parser.add_argument(
    '--model_dir', type=str, default='/tmp/census_model',
    help='Base directory for the model.')

parser.add_argument(
    '--model_type', type=str, default='wide_deep',
    help="Valid model types: {'wide', 'deep', 'wide_deep'}.")

parser.add_argument(
    '--train_epochs', type=int, default=40, help='Number of training epochs.')

parser.add_argument(
    '--epochs_per_eval', type=int, default=2,
    help='The number of training epochs to run between evaluations.')

parser.add_argument(
    '--batch_size', type=int, default=40, help='Number of examples per batch.')

parser.add_argument(
    '--train_data', type=str, default='./labeled_train_X2.csv',
    help='Path to the training data.')

parser.add_argument(
    '--test_data', type=str, default='./labeled_test_X2.csv',
    help='Path to the test data.')

_StoreAction(option_strings=['--test_data'], dest='test_data', nargs=None, const=None, default='./labeled_test_X2.csv', type=<class 'str'>, choices=None, help='Path to the test data.', metavar=None)

In [71]:
_NUM_EXAMPLES = {
    'train': 5901935,
    'validation': 1475484,
}


In [72]:
def build_model_columns():
  """Builds a set of wide and deep feature columns."""
  # Continuous columns
  bd = tf.feature_column.numeric_column('bd')
  #song_length = tf.feature_column.numeric_column('song_length')
  city = tf.feature_column.numeric_column('city')
  language = tf.feature_column.numeric_column('language')
  registered_via = tf.feature_column.numeric_column('registered_via')
  
  '''language = tf.feature_column.categorical_column_with_hash_bucket(
      'language', hash_bucket_size=300)
  city = tf.feature_column.categorical_column_with_hash_bucket(
      'city', hash_bucket_size=2000)'''

  gender = tf.feature_column.categorical_column_with_vocabulary_list(
      'gender', ['male', 'female'])

  #msno = tf.feature_column.categorical_column_with_hash_bucket(
  #    'msno', hash_bucket_size=6000)
  artist_name = tf.feature_column.categorical_column_with_hash_bucket(
      'artist_name', hash_bucket_size=2000)
  composer = tf.feature_column.categorical_column_with_hash_bucket(
      'composer', hash_bucket_size=2000)
  lyricist = tf.feature_column.categorical_column_with_hash_bucket(
      'lyricist', hash_bucket_size=2000)
  # To show an example of hashing:
  source_system_tab = tf.feature_column.categorical_column_with_hash_bucket(
      'source_system_tab', hash_bucket_size=10)
  source_screen_name = tf.feature_column.categorical_column_with_hash_bucket(
      'source_screen_name', hash_bucket_size=10)
  source_type = tf.feature_column.categorical_column_with_hash_bucket(
      'source_type', hash_bucket_size=10)

  # Transformations.
  age_buckets = tf.feature_column.bucketized_column(
      bd, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

  # Wide columns and deep columns.
  base_columns = [
      gender, artist_name, source_system_tab, source_screen_name,
      source_type, composer, lyricist, age_buckets,#msno
  ]

  crossed_columns = [
      tf.feature_column.crossed_column(
          ['artist_name', 'composer', 'lyricist'], hash_bucket_size=10000),
  ]

  wide_columns = base_columns + crossed_columns

  deep_columns = [
      bd,
      # song_length,
      city,
      language,
      registered_via,
  ]

  return wide_columns, deep_columns

In [73]:
def build_estimator(model_dir, model_type):
  """Build an estimator appropriate for the given model type."""
  wide_columns, deep_columns = build_model_columns()
  hidden_units = [100, 75, 50, 25]

  # Create a tf.estimator.RunConfig to ensure the model is run on CPU, which
  # trains faster than GPU for this model.
  run_config = tf.estimator.RunConfig().replace(
      session_config=tf.ConfigProto(device_count={'GPU': 0}))

  if model_type == 'wide':
    return tf.estimator.LinearClassifier(
        model_dir=model_dir,
        feature_columns=wide_columns,
        config=run_config)
  elif model_type == 'deep':
    return tf.estimator.DNNClassifier(
        model_dir=model_dir,
        feature_columns=deep_columns,
        hidden_units=hidden_units,
        config=run_config)
  else:
    return tf.estimator.DNNLinearCombinedClassifier(
        model_dir=model_dir,
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units,
        config=run_config)

In [74]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)

  def parse_csv(value):
    print('Parsing', data_file)
    columns = tf.decode_csv(value, field_delim = ',', record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('target')
    return features, tf.equal(labels, '1')

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)
  
  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5) #skip(7376419)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)

  iterator = dataset.make_one_shot_iterator()
  features, labels = iterator.get_next()
  return features, labels

In [75]:
temp = 0
temp2 = 0
def main(unused_argv):
  global temp
  global temp2
  # Clean up the model directory if present
  shutil.rmtree(FLAGS.model_dir, ignore_errors=True)
  model = build_estimator(FLAGS.model_dir, FLAGS.model_type)

  # Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
  for n in range(2):#range(FLAGS.train_epochs // FLAGS.epochs_per_eval):
    model.train(input_fn=lambda: input_fn(
        FLAGS.train_data, FLAGS.epochs_per_eval, True, FLAGS.batch_size))

    #results = model.evaluate(input_fn=lambda: input_fn(
    #    FLAGS.test_data, 1, False, FLAGS.batch_size))

    # Display evaluation metrics
    print('Results at epoch', (n + 1) * FLAGS.epochs_per_eval)
    print('-' * 60)

    #for key in sorted(results):
    #  print('%s: %s' % (key, results[key]))
  #model.export_savedmodel(export_dir_base='./')
  column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
  df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                            skip_blank_lines=True, keep_default_na=False)#, dtype={'city':str, 'language':str})
  predictions = list(model.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
      x=df, num_epochs=1,shuffle=False)))
  res = pd.DataFrame(predictions)
  temp = model
  temp2 = predictions
  #print(res.count)
  res.to_csv('pred.csv', sep = ',', header = False)
  

if __name__ == '__main__':
  tf.logging.set_verbosity(tf.logging.INFO)
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed + [temp])

INFO:tensorflow:Using config: {'_tf_random_seed': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_session_config': device_count {
  key: "GPU"
}
, '_task_type': 'worker', '_model_dir': '/tmp/census_model', '_task_id': 0, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_service': None, '_keep_checkpoint_max': 5, '_master': '', '_is_chief': True, '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1369bcd68>, '_save_checkpoints_steps': None}
Parsing ./labeled_train_X2.csv
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/census_model/model.ckpt.
INFO:tensorflow:loss = 29.676, step = 1
INFO:tensorflow:global_step/sec: 110.706
INFO:tensorflow:loss = 27.6444, step = 101 (0.906 sec)
INFO:tensorflow:global_step/sec: 195.546
INFO:tensorflow:loss = 27.4404, step = 201 (0.511 sec)
INFO:tensorflow:global_step/sec: 120.848
INFO:tensor

INFO:tensorflow:global_step/sec: 119.289
INFO:tensorflow:loss = 24.5357, step = 7801 (0.838 sec)
INFO:tensorflow:global_step/sec: 200.839
INFO:tensorflow:loss = 25.5475, step = 7901 (0.498 sec)
INFO:tensorflow:global_step/sec: 121.74
INFO:tensorflow:loss = 29.6367, step = 8001 (0.822 sec)
INFO:tensorflow:global_step/sec: 118.647
INFO:tensorflow:loss = 29.8523, step = 8101 (0.843 sec)
INFO:tensorflow:global_step/sec: 239.605
INFO:tensorflow:loss = 26.3702, step = 8201 (0.417 sec)
INFO:tensorflow:global_step/sec: 239.113
INFO:tensorflow:loss = 25.4347, step = 8301 (0.418 sec)
INFO:tensorflow:global_step/sec: 98.4119
INFO:tensorflow:loss = 28.9713, step = 8401 (1.017 sec)
INFO:tensorflow:global_step/sec: 237.946
INFO:tensorflow:loss = 27.9344, step = 8501 (0.420 sec)
INFO:tensorflow:global_step/sec: 134.758
INFO:tensorflow:loss = 26.7604, step = 8601 (0.742 sec)
INFO:tensorflow:global_step/sec: 110.373
INFO:tensorflow:loss = 27.9264, step = 8701 (0.906 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 146.454
INFO:tensorflow:loss = 23.765, step = 16201 (0.683 sec)
INFO:tensorflow:global_step/sec: 118.588
INFO:tensorflow:loss = 24.6662, step = 16301 (0.843 sec)
INFO:tensorflow:global_step/sec: 165.997
INFO:tensorflow:loss = 24.1476, step = 16401 (0.602 sec)
INFO:tensorflow:global_step/sec: 153.414
INFO:tensorflow:loss = 24.4397, step = 16501 (0.652 sec)
INFO:tensorflow:global_step/sec: 157.104
INFO:tensorflow:loss = 28.9844, step = 16601 (0.637 sec)
INFO:tensorflow:global_step/sec: 128.634
INFO:tensorflow:loss = 26.2719, step = 16701 (0.779 sec)
INFO:tensorflow:global_step/sec: 119.557
INFO:tensorflow:loss = 26.2927, step = 16801 (0.834 sec)
INFO:tensorflow:global_step/sec: 110.763
INFO:tensorflow:loss = 28.9583, step = 16901 (0.903 sec)
INFO:tensorflow:global_step/sec: 231.84
INFO:tensorflow:loss = 25.1676, step = 17001 (0.431 sec)
INFO:tensorflow:global_step/sec: 127.478
INFO:tensorflow:loss = 24.8437, step = 17101 (0.785 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 187.227
INFO:tensorflow:loss = 26.774, step = 24601 (0.535 sec)
INFO:tensorflow:global_step/sec: 205.341
INFO:tensorflow:loss = 25.4104, step = 24701 (0.486 sec)
INFO:tensorflow:global_step/sec: 169.225
INFO:tensorflow:loss = 25.9841, step = 24801 (0.592 sec)
INFO:tensorflow:global_step/sec: 182.851
INFO:tensorflow:loss = 25.393, step = 24901 (0.547 sec)
INFO:tensorflow:global_step/sec: 183.275
INFO:tensorflow:loss = 27.6718, step = 25001 (0.546 sec)
INFO:tensorflow:global_step/sec: 178.747
INFO:tensorflow:loss = 26.9055, step = 25101 (0.559 sec)
INFO:tensorflow:global_step/sec: 181.722
INFO:tensorflow:loss = 23.866, step = 25201 (0.554 sec)
INFO:tensorflow:global_step/sec: 165.393
INFO:tensorflow:loss = 24.8469, step = 25301 (0.602 sec)
INFO:tensorflow:global_step/sec: 180.925
INFO:tensorflow:loss = 25.8865, step = 25401 (0.553 sec)
INFO:tensorflow:global_step/sec: 177.231
INFO:tensorflow:loss = 26.0435, step = 25501 (0.564 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 161.705
INFO:tensorflow:loss = 29.1323, step = 33001 (0.619 sec)
INFO:tensorflow:global_step/sec: 78.4497
INFO:tensorflow:loss = 28.2535, step = 33101 (1.275 sec)
INFO:tensorflow:global_step/sec: 144.163
INFO:tensorflow:loss = 25.3846, step = 33201 (0.694 sec)
INFO:tensorflow:global_step/sec: 230.785
INFO:tensorflow:loss = 26.7118, step = 33301 (0.433 sec)
INFO:tensorflow:global_step/sec: 149.148
INFO:tensorflow:loss = 24.8556, step = 33401 (0.670 sec)
INFO:tensorflow:global_step/sec: 98.0342
INFO:tensorflow:loss = 23.6547, step = 33501 (1.020 sec)
INFO:tensorflow:global_step/sec: 128.897
INFO:tensorflow:loss = 27.6072, step = 33601 (0.776 sec)
INFO:tensorflow:global_step/sec: 230.734
INFO:tensorflow:loss = 27.2062, step = 33701 (0.434 sec)
INFO:tensorflow:global_step/sec: 121.285
INFO:tensorflow:loss = 23.8699, step = 33801 (0.824 sec)
INFO:tensorflow:global_step/sec: 81.141
INFO:tensorflow:loss = 24.9732, step = 33901 (1.233 sec)
INFO:tensorflow:globa

...

INFO:tensorflow:loss = 26.0999, step = 713972 (0.703 sec)
INFO:tensorflow:global_step/sec: 140.735
INFO:tensorflow:loss = 23.6603, step = 714072 (0.711 sec)
INFO:tensorflow:global_step/sec: 232.186
INFO:tensorflow:loss = 22.4809, step = 714172 (0.431 sec)
INFO:tensorflow:global_step/sec: 232.941
INFO:tensorflow:loss = 26.3253, step = 714272 (0.429 sec)
INFO:tensorflow:global_step/sec: 142.619
INFO:tensorflow:loss = 27.8832, step = 714372 (0.701 sec)
INFO:tensorflow:global_step/sec: 137.889
INFO:tensorflow:loss = 27.8426, step = 714472 (0.726 sec)
INFO:tensorflow:global_step/sec: 194.666
INFO:tensorflow:loss = 22.0043, step = 714572 (0.513 sec)
INFO:tensorflow:global_step/sec: 177.044
INFO:tensorflow:loss = 24.5732, step = 714672 (0.565 sec)
INFO:tensorflow:global_step/sec: 172.239
INFO:tensorflow:loss = 26.2637, step = 714772 (0.581 sec)
INFO:tensorflow:global_step/sec: 159.322
INFO:tensorflow:loss = 24.5572, step = 714872 (0.627 sec)
INFO:tensorflow:global_step/sec: 231.207
INFO:tenso

INFO:tensorflow:loss = 25.7388, step = 722272 (0.779 sec)
INFO:tensorflow:global_step/sec: 121.679
INFO:tensorflow:loss = 25.2849, step = 722372 (0.824 sec)
INFO:tensorflow:global_step/sec: 182.906
INFO:tensorflow:loss = 24.867, step = 722472 (0.545 sec)
INFO:tensorflow:global_step/sec: 140.561
INFO:tensorflow:loss = 24.548, step = 722572 (0.713 sec)
INFO:tensorflow:global_step/sec: 184.46
INFO:tensorflow:loss = 26.6645, step = 722672 (0.541 sec)
INFO:tensorflow:global_step/sec: 144.194
INFO:tensorflow:loss = 26.2024, step = 722772 (0.694 sec)
INFO:tensorflow:global_step/sec: 230.911
INFO:tensorflow:loss = 26.1133, step = 722872 (0.433 sec)
INFO:tensorflow:global_step/sec: 157.378
INFO:tensorflow:loss = 25.2781, step = 722972 (0.635 sec)
INFO:tensorflow:global_step/sec: 129.679
INFO:tensorflow:loss = 23.9245, step = 723072 (0.772 sec)
INFO:tensorflow:global_step/sec: 129.589
INFO:tensorflow:loss = 25.1316, step = 723172 (0.772 sec)
INFO:tensorflow:global_step/sec: 150.482
INFO:tensorfl

INFO:tensorflow:loss = 27.1419, step = 730572 (0.740 sec)
INFO:tensorflow:global_step/sec: 128.02
INFO:tensorflow:loss = 29.7496, step = 730672 (0.781 sec)
INFO:tensorflow:global_step/sec: 224.059
INFO:tensorflow:loss = 22.1634, step = 730772 (0.447 sec)
INFO:tensorflow:global_step/sec: 148.674
INFO:tensorflow:loss = 23.831, step = 730872 (0.672 sec)
INFO:tensorflow:global_step/sec: 153.494
INFO:tensorflow:loss = 25.1504, step = 730972 (0.652 sec)
INFO:tensorflow:global_step/sec: 133.124
INFO:tensorflow:loss = 24.82, step = 731072 (0.751 sec)
INFO:tensorflow:global_step/sec: 223.654
INFO:tensorflow:loss = 25.0009, step = 731172 (0.447 sec)
INFO:tensorflow:global_step/sec: 109.827
INFO:tensorflow:loss = 25.3104, step = 731272 (0.911 sec)
INFO:tensorflow:global_step/sec: 190.364
INFO:tensorflow:loss = 23.8507, step = 731372 (0.525 sec)
INFO:tensorflow:global_step/sec: 221.393
INFO:tensorflow:loss = 23.8504, step = 731472 (0.452 sec)
INFO:tensorflow:global_step/sec: 143.062
INFO:tensorflo

SystemExit: 

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                        skip_blank_lines=True, keep_default_na=False)
predictions = list(temp.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
  x=df, num_epochs=1,shuffle=False)))
res = pd.DataFrame(predictions)

AttributeError: 'NoneType' object has no attribute 'predict'

[{'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.58246171], dtype=float32),
  'logits': array([ 0.33288729], dtype=float32),
  'probabilities': array([ 0.41753826,  0.58246171], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.41197711], dtype=float32),
  'logits': array([-0.35579801], dtype=float32),
  'probabilities': array([ 0.58802289,  0.41197711], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.42618015], dtype=float32),
  'logits': array([-0.29745328], dtype=float32),
  'probabilities': array([ 0.57381982,  0.42618012], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.31419533], dtype=float32),
  'logits': array([-0.7805779], dtype=float32),
  'probabilities': array([ 0.68580467,  0.31419536], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array

In [ ]:
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)

'''model.train(
    input_fn=lambda: wide_deep.input_fn(
        TEST_CSV, num_epochs=100, shuffle=True, batch_size=3))'''
predictions = model.predict(
    input_fn=lambda: wide_deep.input_fn(
        TEST_CSV, num_epochs=1, shuffle=False))

#predictions = list(model.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predicted_classes))